In [31]:
# install transformers library
#!pip install transformers
#!pip install datasets
#!pip install evaluate


# import libraries
import numpy as np
import pandas as pd
#import torch
import transformers 
from sklearn.model_selection import train_test_split
#from torch.utils.data import DataLoader
from datasets import Dataset
#my_dict = {"a": [1, 2, 3]}
#dataset = Dataset.from_dict(my_dict)

# set device
#device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [32]:

# load your dataset

train_neg = [tweet[:-1] for tweet in open('train_neg.txt').readlines()]
train_pos = [tweet[:-1] for tweet in open('train_pos.txt').readlines()]


In [33]:
# split dataset into train and test sets
X, y = train_neg + train_pos, [0]*100000 + [1]*100000
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

dataset_train = Dataset.from_list([{'label' : y_train[i], 'text':X_train[i]} for i in range(len(y_train))])
dataset_test = Dataset.from_list([{'label' : y_test[i], 'text':X_test[i]} for i in range(len(y_test))])

In [ ]:
from transformers import AutoTokenizer

#MODEL = 'roberta-base'
#MODEL = 'cardiffnlp/twitter-roberta-base-sentiment-latest'
#MODEL = 'distilbert-base-uncased-finetuned-sst-2-english'
#MODEL = 'bert-base-uncased'
#MODEL = 'ProsusAI/finbert'
MODEL = 'finiteautomata/bertweet-base-sentiment-analysis'


tokenizer = AutoTokenizer.from_pretrained(MODEL)


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_dataset_train = dataset_train.map(tokenize_function, batched=True)
tokenized_dataset_test = dataset_test.map(tokenize_function, batched=True)

In [ ]:
from transformers import AutoModelForSequenceClassification

#model = AutoModelForSequenceClassification.from_pretrained("cardiffnlp/twitter-roberta-base-sentiment-latest", num_labels=2, ignore_mismatched_sizes=True)
model = AutoModelForSequenceClassification.from_pretrained(MODEL)

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer")

In [172]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [181]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset_train.shuffle(seed=6).select(range(5000)),
    eval_dataset=tokenized_dataset_test.shuffle(seed=6).select(range(500)),
    compute_metrics=compute_metrics,
)

In [182]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 5000
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1875
  Number of trainable parameters = 134902275


Epoch,Training Loss,Validation Loss,Accuracy
1,0.396100,0.335579,0.866000
2,0.243000,0.494954,0.858000
3,0.140300,0.600331,0.880000


Saving model checkpoint to test_trainer/checkpoint-500
Configuration saved in test_trainer/checkpoint-500/config.json
Model weights saved in test_trainer/checkpoint-500/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 500
  Batch size = 8
Saving model checkpoint to test_trainer/checkpoint-1000
Configuration saved in test_trainer/checkpoint-1000/config.json
Model weights saved in test_trainer/checkpoint-1000/pytorch_model.bin
The following columns in the evaluation set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *

TrainOutput(global_step=1875, training_loss=0.2257679463704427, metrics={'train_runtime': 436.3492, 'train_samples_per_second': 34.376, 'train_steps_per_second': 4.297, 'total_flos': 986675316480000.0, 'train_loss': 0.2257679463704427, 'epoch': 3.0})

In [111]:
pred = trainer.predict((tokenized_dataset_test.shuffle(seed=5).select(range(100))))

The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 100
  Batch size = 8


In [125]:
label_pred = pred.label_ids
pred_label = (pred.predictions[:,1]>pred.predictions[:,0]).astype(int)
#(pred_label==label_real).astype(int).sum()/100

0.84

In [127]:
classification = 2*pred_label-1

In [183]:
TEST = [tweet[:-1] for tweet in open('test_data.txt').readlines()]
XX, yy = TEST, [0]*10000 
DATASET = Dataset.from_list([{'label' : yy[i], 'text':XX[i]} for i in range(len(yy))])
TOK_DATASET = DATASET.map(tokenize_function, batched=True)

  0%|          | 0/10 [00:00<?, ?ba/s]

In [184]:
pred = trainer.predict(TOK_DATASET)

The following columns in the test set don't have a corresponding argument in `RobertaForSequenceClassification.forward` and have been ignored: text. If text are not expected by `RobertaForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 10000
  Batch size = 8


In [185]:
pred_label = (pred.predictions[:,1]>pred.predictions[:,0]).astype(int)
classification = 2*pred_label-1

In [186]:
DF = pd.DataFrame.from_dict({'Id': range(1, 10001), 'Prediction': classification.tolist()})
DF.to_csv('submission_BERT.csv', index = False)